In [1]:
%matplotlib inline
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from plots import *
from kls import *
from torch.distributions.normal import Normal
from torch.distributions.one_hot_categorical import OneHotCategorical as cat
from torch.distributions.beta import Beta
from torch.distributions.uniform import Uniform
from torch.distributions.relaxed_categorical import RelaxedOneHotCategorical as rcat
from torch import logsumexp
import sys
import time
import datetime
import math

In [2]:
Xs = torch.from_numpy(np.load('crosses/obs.npy')).float()
Xs = Xs.transpose(1,0)

N = Xs.shape[0]
D = 2
## Model Parameters
NUM_SAMPLES = 10
NUM_HIDDEN = 64
NUM_STROKES = 2
NUM_LATENTS = 1
NUM_OBS = D
SGD_STEPS = 60000
LEARNING_RATE = 1e-4
CUDA = False
PATH = 'circles'

In [3]:
class Encoder(nn.Module):
    def __init__(self, num_obs= D,
                       num_hidden=NUM_HIDDEN,
                       num_strokes=NUM_STROKES):
        super(self.__class__, self).__init__()
        self.enc_h = nn.Sequential(
            nn.Linear(num_obs, num_hidden),
            nn.Tanh())
        self.enc_log_v = nn.Linear(num_hidden, num_strokes)
        self.enc_log_u_a = nn.Sequential(
            nn.Linear(num_hidden + num_strokes, int(0.5*num_hidden + num_strokes)),
            nn.Tanh(),
            nn.Linear(int(0.5*num_hidden + num_strokes), 1))
        self.enc_log_u_b = nn.Sequential(
            nn.Linear(num_hidden + num_strokes, int(0.5*num_hidden + num_strokes)),
            nn.Tanh(),
            nn.Linear(int(0.5*num_hidden + num_strokes), 1))

    def forward(self, obs, num_samples):
        h = self.enc_h(obs) # (B, H)
        q_v = rcat(temperature=torch.Tensor([0.66]), logits=self.enc_log_v(h))
        v = q_v.sample((num_samples,)) # (S, B, K)
        log_q_v = q_v.log_prob(v)
        h_v = torch.cat((h.repeat(num_samples,1,1), v), -1)
        a = torch.exp(self.enc_log_u_a(h_v)) # (S, B, 1)
        b = torch.exp(self.enc_log_u_b(h_v)) # (S, B, 1)
        q_u = Beta(a, b)
        u = q_u.sample() # (S, B, 1)
        log_q_u = q_u.log_prob(u).sum(-1)
        return u, v, log_q_v +log_q_u
    
    
class Decoder(nn.Module):
    def __init__(self, num_obs=NUM_OBS,
                       num_hidden=NUM_HIDDEN,
                       num_strokes=NUM_STROKES):
        super(self.__class__, self).__init__()

        self.x_sigma = 0.01 * torch.ones((N, num_obs))
        self.dec_mu = nn.Sequential(
            nn.Linear(num_strokes + 1, num_hidden),
            nn.Tanh(),
            nn.Linear(num_hidden, num_obs))

    def forward(self, u, v, obs, num_samples):
        mu = self.dec_mu(torch.cat((u, v), -1)) # (S, B, K+1) -> (S, B, O)
        p_x = Normal(mu, self.x_sigma.repeat(num_samples, 1, 1)) # (S, B, O)
        log_p_x = p_x.log_prob(obs).sum(-1) # (S, B)
        return mu, log_p_x

In [4]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 1e-2)     
        
def initialize():
    enc = Encoder()
    dec = Decoder()
#     enc.apply(weights_init)
    opt1 =  torch.optim.Adam(list(enc.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99)) 
    opt2 = torch.optim.Adam(list(dec.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
    return enc, dec, opt1, opt2
enc, dec, opt1, opt2 = initialize()

In [ ]:
def oneshot(x, N, D, num_samples, num_strokes):
    log_p_xs = torch.zeros((num_samples, N))
    log_prs = torch.zeros((num_samples, N))
    X_mus = torch.zeros((num_samples, N, D))
    u, v, log_q = enc(x, num_samples)
    mu, log_p_x = dec(u, v, x, num_samples)
    pr_u = Uniform(torch.zeros(N, 1), torch.ones(N, 1))
    log_pr_u = pr_u.log_prob(u).sum(-1)
    pr_v = rcat(temperature=torch.Tensor([0.66]), probs=torch.ones(num_strokes)*0.25)
    log_pr_v = pr_v.log_prob(v)
    log_weights = (log_pr_v + log_pr_u + log_p_x - log_q)
    weights = torch.exp(log_weights - logsumexp(log_weights, 0)).detach()
    eubo = torch.mul(weights, log_weights).sum(0).mean()
    elbo = log_weights.mean(0).mean()

    return eubo, elbo, mu, u, v

In [ ]:
ELBOs = []
EUBOs = []
time_start = time.time()

for step in range(SGD_STEPS):
    indices = torch.randperm(N)
    Xs_shuffle = Xs[indices]
    opt1.zero_grad()
    eubo, elbo, mu, u, v = oneshot(Xs_shuffle, N, D, NUM_SAMPLES, NUM_STROKES)
    eubo.backward()
    opt1.step()
    opt2.zero_grad()
    eubo, elbo, mu, u, v = oneshot(Xs_shuffle, N, D, NUM_SAMPLES, NUM_STROKES)
    (-elbo).backward()
    opt2.step()
    
    ELBOs.append(elbo.item())
    EUBOs.append(eubo.item())
    
    if step%100 == 0:
        time_end = time.time()  
        print('SGDstep=%d, EUBO=%.3f, ELBO=%.3f (%ds)' % (step, eubo, elbo, time_end - time_start))
        time_start = time.time()


SGDstep=0, EUBO=-13425.275, ELBO=-15240.665 (0s)
SGDstep=100, EUBO=-12311.730, ELBO=-13923.894 (2s)
SGDstep=200, EUBO=-10989.442, ELBO=-12471.407 (2s)
SGDstep=300, EUBO=-9994.586, ELBO=-11162.174 (2s)
SGDstep=400, EUBO=-9226.421, ELBO=-10162.578 (2s)
SGDstep=500, EUBO=-8639.455, ELBO=-9362.979 (2s)
SGDstep=600, EUBO=-8100.061, ELBO=-8955.390 (2s)
SGDstep=700, EUBO=-7316.659, ELBO=-8524.005 (2s)
SGDstep=800, EUBO=-6036.046, ELBO=-7265.677 (2s)
SGDstep=900, EUBO=-4834.835, ELBO=-5989.471 (2s)
SGDstep=1000, EUBO=-3921.011, ELBO=-5087.961 (2s)
SGDstep=1100, EUBO=-3246.668, ELBO=-4331.450 (2s)
SGDstep=1200, EUBO=-2800.466, ELBO=-3881.411 (2s)
SGDstep=1300, EUBO=-2413.765, ELBO=-3346.759 (2s)
SGDstep=1400, EUBO=-2081.641, ELBO=-3061.458 (2s)
SGDstep=1500, EUBO=-1850.322, ELBO=-2767.894 (2s)
SGDstep=1600, EUBO=-1623.672, ELBO=-2532.014 (2s)
SGDstep=1700, EUBO=-1427.251, ELBO=-2326.191 (2s)
SGDstep=1800, EUBO=-1195.044, ELBO=-2165.363 (2s)
SGDstep=1900, EUBO=-1082.166, ELBO=-2046.026 (2s)
SGDs

In [ ]:
eubo, elbo, mu, u, v = oneshot(Xs, N, D, NUM_SAMPLES)

In [ ]:
vv = v.mean(0).argmax(-1)
plt.plot(vv[:200].data.numpy(), 'ro')

In [ ]:
T = 150
plt.scatter(Xs[:T, 0].data.numpy(), Xs[:T, 1].data.numpy())
plt.xlim([-3,3])
plt.ylim([-3,3])

In [ ]:
for s in range(NUM_SAMPLES):
    fig = plt.figure(figsize=(12,5))
    ax1 = fig.add_subplot(1,2,1)
    ax2 = fig.add_subplot(1,2,2)   
    ax1.scatter(mu[s, :, 0].data.numpy(), mu[s, :, 1].data.numpy())
#     ax2.scatter(Xs_shuffle[:, 0].data.numpy(), Xs_shuffle[:, 1].data.numpy())
    ax1.set_xlim([-3,3])
    ax1.set_ylim([-3,3])


In [ ]:
a = torch.linspace(0.0, 0.2, 100)[:, None]
angles = dec.dec_angles(a) 
rads = dec.dec_rad(a)
x_mus = angles * rads

fig = plt.figure(figsize=(5,5))
ax1 = fig.add_subplot(1,1,1)
ax1.scatter(x_mus[:, 0].data.numpy(), x_mus[:, 1].data.numpy())
ax1.set_xlim([-3,3])
ax1.set_ylim([-3,3])